# <span style="color:#ff5f27;">🛠 BTC timeseries Feature Engineering</span>
---

### <span style="color:#ff5f27;"> 📝 Imports</span>

In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import datetime

### <span style="color:#ff5f27;"> 💽 Loading Data</span>

In [2]:
df = pd.read_csv("btc_raw_600.csv", index_col=0, parse_dates=["date"])

df.head()

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,unix
0,2020-11-27,17149.47,17457.62,16438.08,17139.52,85297.024787,1.446670e+09,1572299,42507.358433,7.209856e+08,1606428000000
1,2020-11-28,17139.53,17880.49,16865.56,17719.85,64910.699970,1.128062e+09,1209385,31785.706898,5.520447e+08,1606514400000
2,2020-11-29,17719.84,18360.05,17517.00,18184.99,55329.016303,9.967438e+08,1141289,27883.985296,5.022877e+08,1606600800000
3,2020-11-30,18185.00,19863.16,18184.99,19695.87,115463.466888,2.205929e+09,1744149,61033.414903,1.166944e+09,1606687200000
4,2020-12-01,19695.87,19888.00,18001.12,18764.96,127698.762652,2.446070e+09,2023802,63805.392898,1.223283e+09,1606773600000


In [3]:
# Because we have tweets(from Kaggle dataset) for this specific timerange, we will cut redundant btc data pieces.
df = df[(df.date >= '2021-02-05 10:00:00') & (df.date <= '2022-06-04 23:00:00')] # hardcoded timerange because of tweets timeranges

df = df.reset_index(drop=True)

In [4]:
df

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,unix
0,2021-02-06,38289.32,40955.51,38215.94,39186.94,98757.311183,3.922095e+09,2291646,52015.513362,2.065181e+09,1612562400000
1,2021-02-07,39181.01,39700.00,37351.00,38795.69,84363.679763,3.256521e+09,1976357,40764.388959,1.574483e+09,1612648800000
2,2021-02-08,38795.69,46794.45,37988.89,46374.87,138597.536914,5.881537e+09,3230961,72345.891568,3.069314e+09,1612735200000
3,2021-02-09,46374.86,48142.19,44961.09,46420.42,115499.861712,5.386255e+09,3119034,57429.564347,2.679451e+09,1612821600000
4,2021-02-10,46420.42,47310.00,43727.00,44807.58,97154.182200,4.431650e+09,2891592,47971.985731,2.190231e+09,1612908000000
...,...,...,...,...,...,...,...,...,...,...,...
479,2022-05-31,31734.23,32399.00,31200.01,31801.04,62433.116320,1.981259e+09,1161340,30046.955110,9.535746e+08,1653944400000
480,2022-06-01,31801.05,31982.97,29301.00,29805.83,103395.633820,3.171191e+09,1404611,47614.812050,1.460767e+09,1654030800000
481,2022-06-02,29805.84,30689.00,29594.55,30452.62,56961.429280,1.711653e+09,1086183,28555.066070,8.581935e+08,1654117200000
482,2022-06-03,30452.63,30699.00,29282.36,29700.21,54067.447270,1.615617e+09,993769,26583.251410,7.943548e+08,1654203600000


In [5]:
df.shape

(484, 11)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         484 non-null    datetime64[ns]
 1   open         484 non-null    float64       
 2   high         484 non-null    float64       
 3   low          484 non-null    float64       
 4   close        484 non-null    float64       
 5   volume       484 non-null    float64       
 6   quote_av     484 non-null    float64       
 7   trades       484 non-null    int64         
 8   tb_base_av   484 non-null    float64       
 9   tb_quote_av  484 non-null    float64       
 10  unix         484 non-null    int64         
dtypes: datetime64[ns](1), float64(8), int64(2)
memory usage: 41.7 KB


---

# <span style="color:#ff5f27;">🛠 Feature Engineering</span>

### <span style="color:#ff5f27;"> 📌 Functions Building</span>

In [7]:
def moving_average(df,window = 7):
    df[f'mean_{window}_days'] = df['close'].rolling(window = window).mean()
    return df

def moving_std(df,window):
    df[f'std_{window}_days'] = df.close.rolling(window = window).std()
    return df

def exponential_moving_average(df, window):
    df[f'exp_mean_{window}_days'] = df.close.ewm(span = window).mean()
    return df

def exponential_moving_std(df, window):
    df[f'exp_std_{window}_days'] = df.close.ewm(span = window).std()
    return df

def momentum_price(df,window):
    '''
     It is the rate of acceleration of a security's price or volume; the speed at which the price is changing.
    '''
    df[f'momentum_{window}_days'] = df.close.diff(window)
    return df

def rate_of_change(df,window):
    '''
     Assets with higher ROC values are considered more likely to be overbought;Lower - likely to be oversold.
    '''
    M = df.close.diff(window - 1)
    N = df.close.shift(window - 1)
    df[f'rate_of_change_{window}_days'] = (M / N) * 100
    return df

def strength_index(df, period):
    '''
     It is a momentum indicator that measures the magnitude of recent price changes
     to evaluate overbought or oversold conditions in the price of a stock or other asset.
     Ranging from [0,100].
     Asset -> 70: asset deemed overbought.
     Asset -> 30: asset getting undersold & undervalued.
    '''
    delta = df.close.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] )
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] )
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com = period-1, adjust = False).mean() / d.ewm(com = period-1, adjust = False).mean()
    df[f'strength_index_{period}_days'] = 100 - 100 / (1 + rs)
    return df

In [8]:
def process_btc_data(df):
    df = moving_average(df,7)
    df = moving_average(df,14)
    df = moving_average(df,56).fillna(0)

    df['signal'] = np.where(df['mean_7_days'] > df['mean_56_days'], 1.0, 0.0)

    for i in [7, 14, 56]:
        for func in [moving_std, exponential_moving_average,
                     exponential_moving_std,
                     momentum_price, rate_of_change,
                     strength_index]:
            df = func(df, i).fillna(0)

    df.date = pd.to_datetime(df.date) 
    return df

### <span style="color:#ff5f27;">〽️ Applying Functions </span>

In [9]:
df_processed = process_btc_data(df)

In [10]:
df_processed.tail(3)

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,...,exp_std_14_days,momentum_14_days,rate_of_change_14_days,strength_index_14_days,std_56_days,exp_mean_56_days,exp_std_56_days,momentum_56_days,rate_of_change_56_days,strength_index_56_days
481,2022-06-02,29805.84,30689.00,29594.55,30452.62,56961.42928,1.711653e+09,1086183,28555.06607,8.581935e+08,...,1679.699391,133.39,4.286187,46.348521,5039.300314,34124.984906,5597.739238,-12991.57,-27.926222,43.640162
482,2022-06-03,30452.63,30699.00,29282.36,29700.21,54067.44727,1.615617e+09,993769,26583.25141,7.943548e+08,...,1576.588412,499.20,0.866529,43.654773,4996.379059,33969.729641,5559.688788,-12551.80,-30.532276,43.052983
483,2022-06-04,29700.21,29988.88,29485.00,29864.04,25617.90113,7.608743e+08,618037,12971.72460,3.853582e+08,...,1472.337144,418.98,-1.419096,44.412343,4925.337066,33825.670351,5514.223362,-12889.93,-29.163058,43.222349


## <span style='color:#ff5f27'> 📥 Save the results</span>

In [11]:
df_processed.to_csv("btc_processed.csv")